In [174]:
import pandas as pd
import sqlite3
import sqlalchemy

In [175]:
conn = sqlite3.connect('ladder.db')

###### 78) Create a daily cumulative sum of the trading volume of the Yum! stock in `yum`.

In [176]:
query = """
select
    date,
    sum(volume) OVER (PARTITION by  date ORDER BY date asc) as daily_cumulative_volume
from yum
limit 5;
"""
pd.read_sql(query, conn)

,date,daily_cumulative_volume
0,2015-01-02,2283400
1,2015-01-05,4418600
2,2015-01-06,5004400
3,2015-01-07,4554100
4,2015-01-08,4258200


###### 79) Create a cumulative sum of the trading volume of Yum! across months. That is, the final row of this query should be the cumulative sum of all months from 2015 through 2019.

In [140]:
query = """
select
    STRFTIME('%Y-%m', date) AS Month,
    SUM(volume) AS daily_cumulative_volume
from yum
where date BETWEEN '2015-01-01' AND '2019-12-31'
group by Month
order by Month
limit 5;
"""
pd.read_sql(query, conn)

,Month,daily_cumulative_volume
0,2015-01,89074400
1,2015-02,98621800
2,2015-03,108827600
3,2015-04,117743300
4,2015-05,131485600


###### 80) For March 2017, create a table from `yum` with the following columns:
* Day of the month
* Row number (ie, the nth trading day of the month)
* Cumulative low (ie, lowest low so far this month)
* Cumulative high (ie, highest high so far this month)
* Cumulative total volume

In [148]:
query = """
select
    STRFTIME('%m-%d', date) as 'Day of the month',
    ROW_NUMBER() OVER (PARTITION BY STRFTIME('%Y-%m', date) ORDER BY date) AS 'Row number',
    MIN(low) OVER (PARTITION BY STRFTIME('%Y-%m', date)) AS 'Cumulative low',
    MAX(high) OVER (PARTITION BY STRFTIME('%Y-%m', date)) AS 'Cumulative high',
    SUM(volume) OVER (PARTITION BY STRFTIME('%Y-%m', date)) AS 'Cumulative total volume'
from yum
where date BETWEEN '2017-03-01' AND '2017-03-31'
limit 5;
"""
pd.read_sql(query, conn)

,Day of the month,Row number,Cumulative low,Cumulative high,Cumulative total volume
0,03-01,1,62.849998,65.949997,65561000
1,03-02,2,62.849998,65.949997,65561000
2,03-03,3,62.849998,65.949997,65561000
3,03-06,4,62.849998,65.949997,65561000
4,03-07,5,62.849998,65.949997,65561000


###### 81) From `yum`, create a column that represents the 7-day moving average of the closing price.

In [147]:
query = """
SELECT
    date,
    close,
    AVG(close) OVER (ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS moving_average
FROM yum
limit 5;
"""
pd.read_sql(query, conn)

,date,close,moving_average
0,2015-01-02,52.012939,52.012939
1,2015-01-05,50.956146,51.484543
2,2015-01-06,50.330696,51.099927
3,2015-01-07,51.998562,51.324586
4,2015-01-08,52.904385,51.640546


###### 82) Repeat the March 2017 problem but instead of cumulative highs, lows, and totals, show the 5-day moving highs and lows. (No need for volume here.)

In [151]:
query = """
SELECT
    date,
    high,
    AVG(high) OVER (ORDER BY date ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS moving_high,
    low,
    AVG(low) over(order by date rows between 5 preceding and current row) as moving_low
FROM yum
limit 5;
"""
pd.read_sql(query, conn)

,date,high,moving_high,low,moving_low
0,2015-01-02,52.846874,52.846874,51.761322,51.761322
1,2015-01-05,52.278934,52.562904,50.948959,51.355141
2,2015-01-06,51.329979,52.151929,49.877785,50.862689
3,2015-01-07,52.056076,52.127966,50.884254,50.868080
4,2015-01-08,53.040977,52.310568,52.365204,51.167505


###### 83) The [**Williams %R**](https://www.investopedia.com/terms/w/williamsr.asp) is an economic trendline indicator of a stock. Query `yum` to only include two columns: the `date` (unmodified), and the 7-day Williams %R of the stock at that date, call it `williams_r`. It is computed as follows:
* Let `h7` be the running 7-day high (ie, highest high of the last 7 trading days).
* Let `l7` be the running 7-day low (ie, lowest low of the lsat 7 trading days).
* `williams_r = (h7 - close) / (h7 - l7)`

The easiest way to do this problem is to make a CTE containing `h7` and `l7`, and then produce `williams_r` by querying your CTE.

In [186]:
c = conn.cursor()
c.execute("DROP VIEW IF EXISTS williams")

c.execute("""
CREATE VIEW williams AS
SELECT
   y.date,
   y.high,
   y.close,
   MAX(y.high) OVER (ORDER BY y.date ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as h7,
   y.low,
   MIN(y.low) OVER (ORDER BY y.date ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as l7
FROM yum y
""")

# Commit the changes
conn.commit()

In [187]:
query = """
select
date,
h7,
l7,
(h7 - close) / (h7 - l7) as williams_r
from williams
limit 5
"""
pd.read_sql(query, conn)

,date,h7,l7,williams_r
0,2015-01-02,52.846874,51.761322,0.768213
1,2015-01-05,52.846874,50.948959,0.996213
2,2015-01-06,52.846874,49.877785,0.847458
3,2015-01-07,52.846874,49.877785,0.285715
4,2015-01-08,53.040977,49.877785,0.043182


###### 84) Next, let's create the [**Stochastic Oscillator**](https://www.investopedia.com/terms/s/stochasticoscillator.asp) of `yum`. The stochastic oscillator is actually two lines: One called _%K_ and the other called _%D_. They are computed as follows:

* Let `h14` and `l14` denote the 14-day highs and lows (similar to last problem)
* `percent_k = (close - l14) / (h14 - l14)`
* `percent_d` is the 3-day moving average of `percent_k`

My solution to this problem involved making _TWO_ CTEs (the second one uses the first one). My query was 29 lines long. As a guide to see if you got the answer correct, here's the first few lines of my (sorted) solution:

In [191]:
c = conn.cursor()
c.execute("DROP VIEW IF EXISTS oscillator")

c.execute("""
CREATE VIEW oscillator AS
SELECT
   y.date,
   y.high,
   y.close,
   MAX(y.high) OVER (ORDER BY y.date ASC ROWS BETWEEN 13 PRECEDING AND CURRENT ROW) as h14,
   y.low,
   MIN(y.low) OVER (ORDER BY y.date ASC ROWS BETWEEN 13 PRECEDING AND CURRENT ROW) as l14
FROM yum y
""")

conn.commit()

In [198]:
c = conn.cursor()
c.execute("DROP VIEW IF EXISTS k_d")

c.execute("""
CREATE VIEW k_d AS
SELECT
   date,
   close,
   ((close - l14) / (h14 - l14)) as percent_k

FROM oscillator
""")

conn.commit()

In [203]:
query = """
select
date,
percent_k,
AVG(percent_k) OVER (ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as percent_d
from k_d
limit 5
"""
pd.read_sql(query, conn)

,date,percent_k,percent_d
0,2015-01-02,0.231787,0.231787
1,2015-01-05,0.003787,0.117787
2,2015-01-06,0.152542,0.129372
3,2015-01-07,0.714285,0.290205
4,2015-01-08,0.956818,0.607882
